In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cx_Oracle

In [2]:
conn = cx_Oracle.connect("hr3", "hoseo","localhost:1521/xe") # 사용자이름, 비번, 서버위치
cursor = conn.cursor()  # cursor : 명령 처리기 

In [19]:
## create table   ## date 설정할때 이름 date로 할수 없음. 명칭 잘 확인 할것!
# cursor.execute("create table navernews(no number primary key, title varchar2(20), link varchar2(50),regdate date, content varchar2(500))")   # oracle create table  

In [20]:
# cursor.execute("create table contact(c_no number primary key,c_name varchar2(50 char) not null,c_phone varchar2(20 char) not null,c_email varchar2(50 char) null,c_regdate date default (sysdate))")

In [3]:
news = pd.read_csv('data-files1/news_크롤링.csv', encoding='utf-8')
news.columns
# news.info()

Index(['Unnamed: 0', '제목', '링크', '날짜', '내용'], dtype='object')

In [25]:
news=news.rename(columns={"Unnamed: 0":"No", "제목":"Title", "링크":"Link" ,"날짜":"RegDate", "내용":"Content"})
# news.head()
news_list=list(news)
news_list[:1]

['No']

In [27]:
cursor.execute("delete from navernews")
sql_query = """INSERT INTO navernews VALUES(:n1 , :n2 , :n3 , :n4 , :n5)"""
for ns in news_list:
    cursor.execute(sql_query ,n1=ns[0], n2=ns[1], n3=ns[2], n4=ns[3], n5=ns[4])

conne.commit()

IndexError: string index out of range